# Synopsis

Here we create a TFIDF matrix from our corpus of novels, and then save a reduced version of this for use in HCA and PCA models. We choose to limit our vocabulary to 1000 top words, based on high-frequency non-stopwords.

We begin by extracting a bag-of-words from the token table. Note that we could have chosen a different set of "bags," e.g. paragraphs or an arbitrary chunk of n tokens.

# Configuration

In [1]:
db_name = 'novels.db'

OHCO = ['genre', 'author', 'book', 'chapter', 'para_num', 'sent_num', 'token_num']
GENRS = OHCO[:1]
AUTHS = OHCO[:2]
BOOKS = OHCO[:3]
CHAPS = OHCO[:4]
PARAS = OHCO[:5]
SENTS = OHCO[:6]

BAG = CHAPS

# Libraries

In [2]:
import sqlite3
import pandas as pd
import numpy as np

# Pragmas

In [3]:
%matplotlib inline

# Process

In [4]:
with sqlite3.connect(db_name) as db:
    tokens = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    vocab = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')
    docs =  pd.read_sql('SELECT * FROM doc', db, index_col=CHAPS)

## Create DTM

### Create word mask

In [5]:
WORDS = (tokens.punc == 0) & (tokens.num == 0)

### Extrct BOW from tokens

To extract a bag-of-words model from our tokens table, we apply a simple `groupby()` operation. Note that we can drop in our hyperparameters easily -- CHAPS and 'term_id' and be replaced. We can easily write a function to simplify this process and make it more configurable. 


In [6]:
BOW = tokens[WORDS].groupby(BAG + ['term_id'])['term_id'].count()

### Convert BOW to DTM

In [7]:
DTM = BOW.unstack().fillna(0)

### Create Bags table

The bags table stores the OHCO content for each doc, since we remove this from the DTM. We can add some stats to this table if we wanted to.

In [8]:
bags = pd.DataFrame(index = DTM.index)
# bags['term_count'] = DTM.sum(1)
# bags['tf'] = bags.term_count / bags.term_count.sum()

In [9]:
DTM = DTM.reset_index(drop=True)
DTM.index.name = 'bag_id'

## Compute Term Frequencies and Weights

### Compute TF

Note that TF is just the term count. It is often normalized in the computing the value, but it is defined as the count in the context of information retrieval.

### Compute IDF

In [10]:
N_docs = DTM.shape[0]
vocab['df'] = DTM[DTM > 0].count()
vocab['idf'] = np.log10(N_docs / vocab.df)

### Test: View most frequent non-stops by IDF

In [11]:
vocab[vocab.stop==0].sort_values('n', ascending=False).head(500)\
    .sort_values('idf', ascending=False).head(20)

,term_str,n,p,port_stem,stop,df,idf
term_id,,,,,,,
14675,manfred,277,0.000185,manfr,0,5,1.806180
21131,scrooge,314,0.000209,scroog,0,5,1.806180
17654,philip,230,0.000153,philip,0,9,1.550907
999,antonia,246,0.000164,antonia,0,9,1.550907
7759,edmund,358,0.000239,edmund,0,10,1.505150
14886,matilda,326,0.000217,matilda,0,13,1.391207
12623,inglethorp,263,0.000175,inglethorp,0,13,1.391207
18037,poirot,386,0.000257,poirot,0,13,1.391207
540,agnes,247,0.000165,agn,0,18,1.249877


### Compute TFIDF

See [Simone Teufel's lectures](https://www.cl.cam.ac.uk/teaching/1415/InfoRtrv/lecture4.pdf)

```
TF: term count
N: number of docs
DF: number of docs with term
log = log10

(1 + log(TF)) * log( N / DF)
```

In [12]:
TFIDF = DTM * vocab['idf']

### Test: Stopwords Detected?

In [13]:
vocab[TFIDF.sum() == 0]

,term_str,n,p,port_stem,stop,df,idf
term_id,,,,,,,
0,a,28533,0.019017,a,1,320,0.0
862,and,44991,0.029986,and,1,320,0.0
1328,as,11252,0.007499,as,1,320,0.0
1987,be,8787,0.005856,be,1,320,0.0
3209,but,9528,0.006350,but,1,320,0.0
3237,by,6923,0.004614,by,1,320,0.0
9681,for,11150,0.007431,for,1,320,0.0
10024,from,6780,0.004519,from,1,320,0.0
11006,had,12858,0.008570,had,1,320,0.0


### Add stats to Vocab

In [14]:
vocab['tfidf_sum'] = TFIDF.sum()
vocab['tfidf_mean'] = TFIDF.mean()
vocab['tfidf_max'] = TFIDF.max()

### Get Top words and Trim Matrix

Basically, implement this SQL query in Pandas:
```
SELECT * 
FROM vocab 
WHERE stop = 0
ORDER BY n DESC
LIMIT 1000
```

In [15]:
def get_top_terms(vocab, no_stops=True, sort_col='n', k=1000):
    if no_stops:
        V = vocab[vocab.stop == 0]
    else:
        V = vocab
    return V.sort_values(sort_col, ascending=False).head(k)

In [16]:
top_n = 1000
TOPV1 = get_top_terms(vocab, sort_col='n')
# TOPV2 = get_top_terms(vocab, sort_col='tfidf_sum')

### Create Reduced TFIDF matrix for later use

In [17]:
tfidf_small = TFIDF[TOPV1.index].stack().to_frame().rename(columns={0:'w'})

# Save data

In [25]:
with sqlite3.connect(db_name) as db:
    vocab.to_sql('vocab', db, if_exists='replace', index=True)
    tokens.to_sql('token', db, if_exists='replace', index=True)
    docs.to_sql('doc', db, if_exists='replace', index=True)
    tfidf_small.to_sql('tfidf_small', db, if_exists='replace', index=True)
    bags.reset_index().to_sql('bag', db, if_exists='replace', index=True, index_label='bag_id')

In [19]:
# END